In [1]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import _obtain_input_shape # this will work for older versions of keras. 2.2.0 or before
#from keras.engine.topology import get_source_inputs
from PIL import Image, UnidentifiedImageError

In [2]:
try :
  %tensorflow_version 2.x
except :
  pass

import tensorflow as tf
print(tf.__version__)

2.12.0


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 

import matplotlib.pyplot as plt

In [4]:
import numpy as np 
import pandas as pd 


In [5]:
# Set the path to the images folder
base_dir = '1math'

# Create subdirectories for training and testing
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Create a list of subdirectories for each digit
digits = ['-','+','=','0','1','2','3','4','5','6','7','8','9','dev']
train_digit_dirs = [os.path.join(train_dir, f'{digit}_jpg') for digit in digits]
validation_digit_dirs = [os.path.join(validation_dir, f'{digit}_jpg') for digit in digits]

# Creating missing directories if they don't exist
for directory in train_digit_dirs + validation_digit_dirs:
    os.makedirs(directory, exist_ok=True)

# Determine the number of instances you want to create for each digit
desired_instances = 10

# Initialize Data Generators
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)



In [6]:
# Get a list of all files in train folders
train_files_list = []
for train_digit_dir in train_digit_dirs:
    train_files = [os.path.join(train_digit_dir, file) for file in os.listdir(train_digit_dir)]
    train_files_list.append(train_files)



In [7]:
from tensorflow.keras.preprocessing import image

# Create a loop to generate additional instances
for digit_files in train_files_list:
    digit_label = os.path.basename(os.path.dirname(digit_files[0]))  # Extracting the digit label from the path
    validation_digit_dir = os.path.join(validation_dir, digit_label)  # Folder for saving augmented copies of this digit
    os.makedirs(validation_digit_dir, exist_ok=True)  # Create a folder if it does not exist
    
    # Processing each file in a loop
    for file_path in digit_files:
        # Skip files .DS_Store
        if file_path.endswith('.DS_Store'):
            continue
        
        try:
            img = image.load_img(file_path, target_size=(150, 150))
            x = image.img_to_array(img)
            x = x.reshape((1,) + x.shape)
            i = 0
            for batch in train_image_generator.flow(x, batch_size=1, save_to_dir=validation_digit_dir, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= desired_instances:
                    break
        except (IOError, SyntaxError, UnidentifiedImageError) as e:
            print(f"Error: {e}. The file {file_path} is not an image. Let's skip.")
            continue
